In [1]:
using CLOUD.ConservationLaws
using CLOUD.SpatialDiscretizations
using CLOUD.Solvers
using CLOUD.InitialConditions
using CLOUD.Mesh
using CLOUD.Analysis
using CLOUD.IO

using OrdinaryDiffEq
using TimerOutputs

#Set up paths
plots_path = "../plots/advection1d_demo/"
results_path = "../results/advection1d_demo/"

#Define the physical problem
a = 1.0  # advection velocity
A = 1.0  # amplitude
L = 1.0  # domain length
k = 2*π/L  # wave number
T = 1*L/a  # end time
initial_data = InitialDataSine(A,k)
conservation_law = linear_advection_equation(a)
u_exact = initial_condition(initial_data, conservation_law)

#Set discretization parameters
M = 10  # number of elements
p = 2  # degree of discretization
N_q = 3  # number of quadrature points
reference_approximation=ReferenceApproximation(
    DGSEM(p), Line(), LGQuadrature())
form = StrongConservationForm()
strategy = Eager()
dt_scale=0.1
mesh = uniform_periodic_mesh(reference_approximation.reference_element, 
    (0.0,L), M)

#Construct spatial discretization
spatial_discretization = SpatialDiscretization(mesh, reference_approximation)
results_path = save_project(conservation_law,
     spatial_discretization, initial_data, form, 
     (0.0, T), strategy, results_path)

┌ Info: Precompiling CLOUD [fb992021-99c7-4c2d-a14b-5e48ac4045b2]
└ @ Base loading.jl:1342


"../results/advection1d_demo_15/"

In [5]:
#run the solver
project = load_project(results_path)
ode_problem = semidiscretize(project["conservation_law"], 
    project["spatial_discretization"], project["initial_data"], project["form"],
    project["tspan"], project["strategy"])
save_solution(ode_problem.u0, project["tspan"][1], results_path, 0)
sol = solve(ode_problem, RK4(), adaptive=false, 
    dt=dt_scale*(L/M)/(a*(2*p+1)), save_everystep=false,
    callback=save_callback(results_path, 10))
save_solution(last(sol.u), last(sol.t), results_path, "final")

In [ ]:
#visualize the solution
plotter = Plotter(spatial_discretization, plots_path)
error_analysis = ErrorAnalysis(spatial_discretization, results_path)
visualize(last(sol.u), plotter,
    "final.pdf", exact_solution=u_exact,
     label="U^h(x,t)", label_exact="U(x,t)")
print("L2 error: ", analyze(error_analysis, last(sol.u), u_exact))